In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn import datasets

In [2]:
iris = datasets.load_iris()

In [3]:
x = iris.data
y = iris.target

In [4]:
# one hot encoding
y = np.eye(len(set(y)))[y]

In [5]:
# normalize
x = (x - x.min(axis=0)) / x.ptp(axis=0)

In [25]:
# split train and test data
train_indices = np.random.choice(len(x), round(len(x)*0.8), replace=False)
test_indices = np.array(list(set(range(len(x))) - set(train_indices)))

x_train = tf.Variable(x[train_indices], tf.float32)
y_train = tf.Variable(y[train_indices], tf.int32)
x_test = tf.Variable(x[test_indices], tf.float32)
y_test = tf.Variable(y[test_indices], tf.int32)

In [12]:
# 4 features, 3 classes, 120 training samples
print(x_train.shape)
print(y_train.shape)

(120, 4)
(120, 3)


In [13]:
# nearest neighbours
k = 9

In [38]:
x_train.shape

TensorShape([120, 4])

In [67]:
print(x_test.shape)
print(x_train.shape)

(30, 4)
(120, 4)


In [76]:
def prediction(x_train, y_train, x_test, k):
    d0 = tf.expand_dims(x_test, axis=1)
    d1 = tf.subtract(x_train, d0)

    print(d0.shape)
    print(d1.shape)
    distance = tf.reduce_sum(tf.abs(d1), axis=2)
    
    _, top_k_indices = tf.nn.top_k(tf.negative(distance), k=k)
    top_k_labels = tf.gather(y_train, top_k_indices)

    sum_predictions = tf.reduce_sum(top_k_labels, axis=1)
    predictions = tf.argmax(sum_predictions, axis=1)
    return predictions

In [77]:
# kNN has no training part
# computing the accuracy from test set
pred = tf.dtypes.cast(prediction(x_train, y_train, x_test, k), tf.int32)
actual = tf.argmax(y_test, axis=1, output_type=tf.int32)

correct_count = tf.math.reduce_sum(tf.dtypes.cast(tf.math.equal(pred, actual), tf.int32))
accuracy = correct_count / y_test.shape[0] * 100
print("Accuracy = ",accuracy.numpy(),"%")

(30, 1, 4)
(30, 120, 4)
tf.Tensor(
[[[-0.44444444  0.125      -0.6779661  -0.79166667]
  [-0.22222222 -0.04166667 -0.23728814 -0.33333333]
  [-0.41666667 -0.16666667 -0.38983051 -0.45833333]
  ...
  [-0.16666667  0.         -0.08474576  0.125     ]
  [ 0.11111111  0.         -0.13559322 -0.29166667]
  [-0.16666667 -0.08333333 -0.27118644 -0.375     ]]

 [[ 0.02777778 -0.04166667  0.05084746  0.        ]
  [ 0.25       -0.20833333  0.49152542  0.45833333]
  [ 0.05555556 -0.33333333  0.33898305  0.33333333]
  ...
  [ 0.30555556 -0.16666667  0.6440678   0.91666667]
  [ 0.58333333 -0.16666667  0.59322034  0.5       ]
  [ 0.30555556 -0.25        0.45762712  0.41666667]]

 [[-0.05555556 -0.125       0.01694915  0.        ]
  [ 0.16666667 -0.29166667  0.45762712  0.45833333]
  [-0.02777778 -0.41666667  0.30508475  0.33333333]
  ...
  [ 0.22222222 -0.25        0.61016949  0.91666667]
  [ 0.5        -0.25        0.55932203  0.5       ]
  [ 0.22222222 -0.33333333  0.42372881  0.41666667]]

 ...
